In [1]:
import requests

In [2]:
import geopandas as gpd
import pandas as pd

## Open State data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Get Columm Data

In [6]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/subject/variables.json")
columns_obj = r.json()

In [7]:
columns_to_replace = {
    key: val["label"]
    for key, val in columns_obj["variables"].items()
    if key.startswith("S0802")
}

In [8]:
mean_time_cols_dict = {
    col_name: col_dict
    for col_name, col_dict in columns_obj["variables"].items()
    if col_dict["label"].split("!!")[-1] == "60 or more minutes"
    and col_name.startswith("S0802")
}

In [9]:
mean_time_cols_dict

{'S0802_C03_089E': {'label': 'Estimate!!Car, truck, or van -- carpooled!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes',
  'concept': 'Means of Transportation to Work by Selected Characteristics',
  'predicateType': 'float',
  'group': 'S0802',
  'limit': 0,
  'attributes': 'S0802_C03_089EA,S0802_C03_089M,S0802_C03_089MA'},
 'S0802_C05_089E': {'label': 'Estimate!!Worked from home!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes',
  'concept': 'Means of Transportation to Work by Selected Characteristics',
  'predicateType': 'int',
  'group': 'S0802',
  'limit': 0,
  'attributes': 'S0802_C05_089EA,S0802_C05_089M,S0802_C05_089MA'},
 'S0802_C02_089E': {'label': 'Estimate!!Car, truck, or van -- drove alone!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes',
  'concept': 'Means of Transportation to Work by Selected Characteristics',
  'predicateType': 'fl

### Get columns to query and rename for later

In [10]:
columns = ["GEO_ID", "S0802_C01_089E"]
columns_formatted = ",".join(columns)

In [11]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5/subject?get={columns_formatted}&for=state:*"
)

In [12]:
response

<Response [200]>

In [13]:
data = response.json()
columns = data[0]
rows = data[1:]
commuting_df = pd.DataFrame(rows, columns=columns)

In [14]:
commuting_df = commuting_df[columns]
commuting_df[columns[1:]] = commuting_df[columns[1:]].astype(float)

In [15]:
columns_to_replace["GEO_ID"] = "AFFGEOID"
commuting_df = commuting_df.rename(columns=columns_to_replace)

In [16]:
commuting_df.columns

Index(['AFFGEOID',
       'Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes',
       'state'],
      dtype='object')

## Merge Data

In [17]:
commuting_gdf = state_gdf.merge(commuting_df, on="AFFGEOID", how="inner")

In [20]:
commuting_gdf = commuting_gdf.to_crs(9311)
commuting_gdf.to_file("data/Commute_More_60_Minutes_Per_State.gpkg")

In [21]:
commuting_gdf.sort_values(
    "Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes"
)

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,Estimate!!Total!!Workers 16 years and over who did not work from home!!TRAVEL TIME TO WORK!!60 or more minutes,state
14,46,01785534,0400000US46,46,SD,South Dakota,00,196346981786,3382720225,"POLYGON ((-318857.556 7726.411, -318299.501 77...",3.3,46.0
10,31,01779792,0400000US31,31,NE,Nebraska,00,198956658395,1371829134,"POLYGON ((-339253.738 -417387.502, -339106.319...",3.5,31.0
26,20,00481813,0400000US20,20,KS,Kansas,00,211755344060,1344141205,"POLYGON ((-174863.27 -553289.527, -163353.728 ...",3.6,20.0
32,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((274418.778 -245971.628, 274712.711 -...",4.1,19.0
47,38,01779797,0400000US38,38,ND,North Dakota,00,178707534813,4403267548,"POLYGON ((-296121.109 437082.483, -296107.817 ...",4.7,38.0
22,49,01455989,0400000US49,49,UT,Utah,00,212886221680,6998824394,"POLYGON ((-1233486.447 -719056.636, -1233235.8...",4.7,49.0
31,30,00767982,0400000US30,30,MT,Montana,00,376962738765,3869208832,"POLYGON ((-1172889.072 508069.731, -1172583.63...",4.8,30.0
45,55,01779806,0400000US55,55,WI,Wisconsin,00,140290039723,29344951758,"MULTIPOLYGON (((1013687.487 121601.135, 101375...",4.8,55.0
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-268912.756 -936547.904, -268600.891...",5.0,40.0
27,02,01785533,0400000US02,02,AK,Alaska,00,1478839695958,245481577452,"MULTIPOLYGON (((-4288407.995 3426181.085, -428...",5.0,2.0
